In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler #scaling de los datos entre 0 y 1
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
data_msal = 'Datos2/Covid19Casos.csv'
data_mobility = 'Datos2/Global_Mobility_Report.csv'

# funciones auxiliares (armadas para mi tesis)

In [3]:
def new_cases_per_day(fecha_sintomas,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -graph: defaults to False. if True, makes graph of new cases per day 
    -ret: defaults to False. if True, returns days-cases per day  np.array casos_por_dia
    given the days of beginning of symptoms, processes to get np.array of cases per day and days since epidemic
    can graph and return this vector with the defaults explained before
    latest update = checked the days again, it works perfectly splendid
    """
    counts = np.bincount(fecha_sintomas)
    ultima_fecha = max(fecha_sintomas)
    aux = range(ultima_fecha+1)
    casos_por_dia = np.vstack((aux,counts[aux])).T #(days since inicio_epidemia x (fecha_inicio_sintomas == days))
    if graph == True:
        plt.xlabel("Dia")
        plt.ylabel("Casos")
        plt.axvspan(ultima_fecha-10, ultima_fecha+3, facecolor='r', alpha=0.5,label="últimos 10 días")
        plt.plot(casos_por_dia[:,0],casos_por_dia[:,1],'-ob',label="casos hasta dia: "+str(ultima_fecha))
        plt.style.use('ggplot')
        plt.legend()
        #plt.savefig("casos_por_dia_bariloche.pdf")
    if ret == True:
        return casos_por_dia   

In [4]:
def cumulative_cases(fecha_sintomas,graph=False,ret=False):
    """
    args: 
    -fecha_sintomas: day of beginning of symptomps for each case, can't be NaN or NaT (already processed)
    -graph: defaults to False. if True, makes graph of new cases per day 
    -ret: defaults to False. if True, returns days-cumulative cases per day np array
    given the days of beginning of symptoms, processes to get np.array of cumulative cases per day and days since epidemic
    can graph and return this vector with the defaults explained before
    latest update = checked the days again, it works perfectly splendid
    """
    ultima_fecha = max(fecha_sintomas)
    casos_por_dia = new_cases_per_day(fecha_sintomas,ret=True)
    casos_acumulados_por_dia = np.copy(casos_por_dia)
    casos_acumulados_por_dia[:,1] = np.cumsum(casos_por_dia[:,1]) 
    if graph == True:
        plt.xlabel("Dia")
        plt.ylabel("Casos acumulados")
        plt.axvspan(ultima_fecha-10, ultima_fecha+3, facecolor='r', alpha=0.5,label="últimos 10 días")
        plt.scatter(casos_acumulados_por_dia[:,0],casos_acumulados_por_dia[:,1],c='b',label="casos acumulados hasta dia: "+str(ultima_fecha))
        plt.legend()
        plt.style.use('ggplot')
        plt.savefig("casos_acumulados_bariloche.pdf")
    if ret == True:
        return casos_acumulados_por_dia 

In [5]:
def change_dates(df):
    #changes fecha_inicio_sintomas according to new criteria
    filt_df1 = (df.fecha_inicio_sintomas.isnull()) #filter fecha_inicio_sintomas = inexistant
    df_sin_fecha = df.loc[filt_df1]
    n_size = df_sin_fecha.shape[0]
    df.loc[filt_df1,"fecha_inicio_sintomas"] = df.loc[filt_df1,"fecha_apertura"] - np.random.randint(0,9,n_size)

# ejercicio en si // trato de hacer todo primero para capital 

# primero manejo los datos de movilidad

In [163]:
data1 = pd.read_csv(data_mobility,sep=",",quotechar='"',dtype={'iso_3166_2_code':str},
                   parse_dates=["date"],na_values=['']) #data loading
df1 = pd.DataFrame(data1)

C:\Users\denis\Anaconda3\envs\tf-gpu\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [164]:
df1 = df1[df1.iso_3166_2_code == 'AR-C']

In [165]:
df1 = df1.drop(['country_region_code','country_region','sub_region_1','sub_region_2','metro_area','iso_3166_2_code','census_fips_code'],axis=1)

In [166]:
primer_dia = min(df1["date"])
ult_dia = max(df1["date"])
print('primer dia',primer_dia)
print('ult dia',ult_dia)

primer dia 2020-02-15 00:00:00
ult dia 2020-11-13 00:00:00


# ahora manejo los datos del msal

In [10]:
data2 = pd.read_csv(data_msal,sep=",",quotechar='"',
                   parse_dates=["fecha_inicio_sintomas","fecha_apertura","fecha_internacion","fecha_cui_intensivo",
                               "fecha_fallecimiento","fecha_diagnostico","ultima_actualizacion"]
                    ,na_values=['']) #data loading

In [11]:
df2 = pd.DataFrame(data2)

In [12]:
df2 = df2[df2.residencia_provincia_nombre == 'CABA']

In [13]:
df2 = df2.drop(["id_evento_caso","sexo","edad","edad_años_meses","residencia_pais_nombre",
          "residencia_provincia_nombre","residencia_departamento_nombre",
         "carga_provincia_nombre","carga_provincia_id","origen_financiamiento",
         "residencia_provincia_id","residencia_departamento_id","ultima_actualizacion",
        "asistencia_respiratoria_mecanica","sepi_apertura"],axis=1)

In [14]:
df2

,fecha_inicio_sintomas,fecha_apertura,fecha_internacion,cuidado_intensivo,fecha_cui_intensivo,fallecido,fecha_fallecimiento,clasificacion,clasificacion_resumen,fecha_diagnostico
0,NaT,2020-06-01,NaT,NO,NaT,NO,NaT,Caso Descartado,Descartado,2020-06-09
6,2020-05-31,2020-06-01,NaT,NO,NaT,NO,NaT,Caso Descartado,Descartado,2020-06-01
8,NaT,2020-06-01,NaT,NO,NaT,NO,NaT,Caso confirmado por laboratorio - No activo (p...,Confirmado,2020-06-01
10,2020-05-26,2020-06-01,2020-05-31,NO,NaT,NO,NaT,Caso confirmado por laboratorio - No Activo po...,Confirmado,2020-05-31
13,2020-05-18,2020-06-01,NaT,NO,NaT,NO,NaT,Caso confirmado por laboratorio - No activo (p...,Confirmado,2020-06-01
...,...,...,...,...,...,...,...,...,...,...
3283418,2020-05-15,2020-06-01,NaT,NO,NaT,NO,NaT,Caso Descartado,Descartado,2020-06-01
3283422,NaT,2020-06-01,NaT,NO,NaT,NO,NaT,Caso Descartado,Descartado,2020-06-02
3283428,2020-05-28,2020-06-01,2020-05-31,NO,NaT,NO,NaT,Caso confirmado por laboratorio - No Activo po...,Confirmado,2020-06-01
3283433,NaT,2020-06-01,NaT,NO,NaT,NO,NaT,Caso Descartado,Descartado,2020-06-01


# primero lo convencional, casos en funcion del tiempo

In [15]:
#datos acumulados - fecha 
df = df2.copy()
df = df[df.clasificacion_resumen == "Confirmado"]

In [16]:
df = df.drop(["fecha_internacion","cuidado_intensivo","fecha_cui_intensivo","fallecido","fecha_fallecimiento","clasificacion","clasificacion_resumen","fecha_diagnostico"],axis=1)

In [17]:
filt_df1 = (df.fecha_inicio_sintomas.notnull()) #filter fecha_inicio_sintomas = inexistant
inicio_epidemia = min(df.loc[filt_df1,"fecha_inicio_sintomas"]) #first symptoms of a person registered
ultima_actualizacion_sintomas = max(df.loc[filt_df1,"fecha_inicio_sintomas"]) #last day symptoms of a person registered
ultima_actualizacion_apertura = max(df["fecha_apertura"])
df["fecha_inicio_sintomas"] -= inicio_epidemia #correcting by inicio_epidemia 
df["fecha_apertura"] -= inicio_epidemia #correcting by inicio_epidemia
df.fecha_inicio_sintomas = df.fecha_inicio_sintomas.dt.days #change to int, ditch days 
df.fecha_apertura = df.fecha_apertura.dt.days #change to int, ditch days
change_dates(df) #replaces non existing fecha_inicio_sintomas acc to new criteria
print("primer sintoma de persona confirmada: ",inicio_epidemia)
print("ultimo sintoma de persona confirmada: ",ultima_actualizacion_sintomas)
print("ultima apertura de persona confirmada: ",ultima_actualizacion_apertura)

primer sintoma de persona confirmada:  2020-01-06 00:00:00
ultimo sintoma de persona confirmada:  2020-11-15 00:00:00
ultima apertura de persona confirmada:  2020-11-16 00:00:00


In [18]:
fecha_sintomas = df.fecha_inicio_sintomas.to_numpy() #numpy array of fecha_inicio_sintomas
fecha_apertura = df.fecha_apertura.to_numpy() #numpy array of fecha_apertura
fecha_sintomas = fecha_sintomas.astype(int)

In [19]:
ac_cases = cumulative_cases(fecha_sintomas,graph=False,ret=True) #casos acumulados 
dif_cases = np.zeros((ac_cases.shape[0],1),dtype=float)

In [20]:
for i in range(ac_cases.shape[0]):
    if i != 0:
        dif_cases[i] = ac_cases[i,1] - ac_cases[i-1,1]
ac_cases = np.concatenate([ac_cases,dif_cases],axis=1)

# fallecimientos en funcion del tiempo 

In [21]:
dff = df2.copy()
dff = dff[dff.clasificacion_resumen == "Confirmado"]
dff = dff[dff.fallecido == "SI"]

In [22]:
dff = dff.drop(["fecha_inicio_sintomas","fecha_apertura","fecha_internacion","cuidado_intensivo","fecha_cui_intensivo","fallecido",
         "clasificacion","clasificacion_resumen","fecha_diagnostico"],axis=1)

In [23]:
dff

,fecha_fallecimiento
117,2020-06-17
282,2020-06-09
324,2020-06-16
481,2020-06-23
1129,2020-06-03
...,...
3281333,2020-07-08
3281713,2020-06-19
3281907,2020-06-06
3282167,2020-06-16


In [24]:
dff["fecha_fallecimiento"] -= inicio_epidemia

In [25]:
dff

,fecha_fallecimiento
117,163 days
282,155 days
324,162 days
481,169 days
1129,149 days
...,...
3281333,184 days
3281713,165 days
3281907,152 days
3282167,162 days


In [26]:
dff.fecha_fallecimiento = dff.fecha_fallecimiento.dt.days
fecha_fallecimiento = dff.fecha_fallecimiento.to_numpy() #numpy array of fecha_apertura
fecha_fallecimiento = fecha_fallecimiento.astype(int)

In [27]:
fac_cases = cumulative_cases(fecha_fallecimiento,graph=False,ret=True)
fac_cases = fac_cases[:,1].reshape(-1,1)

In [28]:
ac_cases = np.concatenate([ac_cases,fac_cases],axis=1)

In [29]:
dif_cases = np.zeros((ac_cases.shape[0],1),dtype=float)
for i in range(ac_cases.shape[0]):
    if i != 0:
        dif_cases[i] = ac_cases[i,3] - ac_cases[i-1,3]
ac_cases = np.concatenate([ac_cases,dif_cases],axis=1)

In [30]:
ac_cases

array([[0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [1.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       [2.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00],
       ...,
       [3.13000e+02, 1.53027e+05, 4.60000e+01, 5.24100e+03, 8.00000e+00],
       [3.14000e+02, 1.53058e+05, 3.10000e+01, 5.24400e+03, 3.00000e+00],
       [3.15000e+02, 1.53069e+05, 1.10000e+01, 5.24700e+03, 3.00000e+00]])

# agrego la poblacion de caba

In [32]:
filas = ac_cases.shape[0]
poblacion = 3075646
pop = np.zeros((filas,1)) + poblacion
ac_cases = np.concatenate([ac_cases,pop],axis=1)

# descartados

In [95]:
#datos acumulados - fecha 
dff = df2.copy()
dff = dff[dff.]
dff = dff[dff.clasificacion_resumen == "Descartado"]

In [96]:
dff = dff.drop(["fecha_inicio_sintomas","fecha_apertura","fecha_internacion","cuidado_intensivo","fecha_cui_intensivo",
                "fallecido","fecha_fallecimiento","clasificacion","clasificacion_resumen"],axis=1)

In [97]:
dff["fecha_diagnostico"] -= inicio_epidemia
dff.fecha_diagnostico = dff.fecha_diagnostico.dt.days
fecha_diagnostico = dff.fecha_diagnostico.to_numpy() #numpy array of fecha_apertura
fecha_diagnostico = fecha_diagnostico.astype(int)
fecha_diagnostico[fecha_diagnostico<0] = 0

In [100]:
fac_cases = cumulative_cases(fecha_diagnostico,graph=False,ret=True)
fac_cases = fac_cases[:,1].reshape(-1,1)

In [101]:
ac_cases = np.concatenate([ac_cases,fac_cases],axis=1)

# cuidados intensivos 

In [119]:
dff = df2.copy()
dff = dff[dff.clasificacion_resumen == "Confirmado"]
dff = dff[dff.cuidado_intensivo=="SI"] 
dff = dff.drop(["fecha_inicio_sintomas","fecha_apertura","fecha_internacion","cuidado_intensivo","fallecido","fecha_fallecimiento",
         "clasificacion","clasificacion_resumen","fecha_diagnostico"],axis=1)

In [122]:
dff["fecha_cui_intensivo"] -= inicio_epidemia

In [123]:
dff.fecha_cui_intensivo = dff.fecha_cui_intensivo.dt.days
fecha_cui_intensivo = dff.fecha_cui_intensivo.to_numpy() #numpy array of fecha_apertura
fecha_cui_intensivo = fecha_cui_intensivo.astype(int)

In [124]:
fac_cases = cumulative_cases(fecha_fallecimiento,graph=False,ret=True)
fac_cases = fac_cases[:,1].reshape(-1,1)

In [127]:
ac_cases = np.concatenate([ac_cases,fac_cases],axis=1)

# le sumamos los datos de movilidad

In [167]:
df1["date"] -= inicio_epidemia

In [168]:
df1.date = df1.date.dt.days

In [171]:
min_date = min(df1.date)
max_date = max(df1.date)

In [172]:
df1 = df1.to_numpy()

In [176]:
df1 = df1[:,1:]

In [180]:
indices1 = (min_date<=ac_cases[:,0])
indices2 = (ac_cases[:,0]<=max_date)
indices = indices1 * indices2
true_data_msal = ac_cases[indices]

In [184]:
ac_cases.shape

(316, 8)

In [185]:
true_data_msal.shape

(273, 8)

In [186]:
df1.shape

(273, 6)

In [188]:
total = np.concatenate([true_data_msal,df1],axis=1)

In [189]:
total.shape

(273, 14)

In [192]:
header="fecha,"
np.savetxt('caba.txt',total,delimiter=",")